In [1]:
import OpenBlender
import pandas as pd    
import json    
import numpy as np    
import csv    
from sklearn.mixture import GaussianMixture    
from sklearn.linear_model import LogisticRegression    
from sklearn import preprocessing    
from sklearn.model_selection import GridSearchCV    
from sklearn.model_selection import RandomizedSearchCV    
from sklearn.preprocessing import MinMaxScaler    
from sklearn.preprocessing import MaxAbsScaler    
from sklearn.preprocessing import StandardScaler    
from sklearn.preprocessing import RobustScaler    
from sklearn.preprocessing import Normalizer    
from sklearn.preprocessing import QuantileTransformer    
from sklearn.preprocessing import PowerTransformer    
from sklearn.feature_selection import SelectFromModel    
from sklearn.metrics import silhouette_samples, silhouette_score    
from sklearn.cluster import KMeans    
from sklearn.feature_selection import VarianceThreshold    
from time import localtime    
import datetime    
from datetime import datetime as dt    
from matplotlib import pyplot as plt    
from scipy.spatial.distance import cdist  

In [2]:
def retTStamp(l):       
        ret = []       
        for i in l:
            try:
                date_time_obj = datetime.datetime.strptime(i, '%Y-%m-%d %H:%M:%S').replace(minute=0,second=0,hour=0)
            except:
                hold = i.split()
                date = hold[0].split("/")
                time = hold[1].split(":")
                date_time_obj = datetime.datetime(int('20'+str(date[2])),int(date[0]),int(date[1]),int(time[0]),int(time[1]),0).replace(minute=0,hour=0)

            ret.append(date_time_obj)       
        return ret    

In [72]:
data       = pd.read_csv('input/coronaUTD.csv')      
input_data = pd.read_csv('input/csse_covid_19_data/csse_covid_19_daily_reports/05-10-2020.csv')      





# reformatting to be the same as current data structure      
input_data.insert(0,'confirmed',input_data['Confirmed'])      
input_data.insert(1,'countryregion',input_data['Country_Region'])      
input_data.insert(2,'deaths',input_data['Deaths'])      
input_data.insert(3,'latitude',input_data['Lat'])      
input_data.insert(4,'longitude',input_data['Long_'])      
input_data.insert(5,'provincestate',input_data['Province_State'])      
input_data.insert(6,'recovered',input_data['Recovered'])      


datetime_conver = retTStamp(input_data['Last_Update'])      
input_data.insert(7,'timestamp',datetime_conver)      
input_data['timestamp'] = input_data.timestamp.values.astype(np.int64) // 10 ** 9      

for i,j in input_data.iterrows():      
    if(not type(j['provincestate'])==str):      
        input_data.at[i,'provincestate'] = j['countryregion']      
        
input_data.drop(['FIPS','Admin2','Active','Combined_Key','Confirmed','Country_Region','Deaths','Lat','Long_','Province_State','Recovered','Last_Update'],axis=1,inplace=True)      



distinctDates = list(set(list(input_data['timestamp'])))      
distinctPS    = list(set(list(input_data['provincestate'])))      

dataHold      = []       
for date in distinctDates:      
    currData = input_data[input_data['timestamp'] == date]      
    for ps in distinctPS:      
        specificData   = currData[currData['provincestate'] == ps]      
        if(len(specificData) == 0):      
            continue      
        else:      
            totalConfirmed = sum(list(specificData['confirmed']))      
            totalDeaths    = sum(list(specificData['deaths']))      
            totalrecovered = sum(list(specificData['recovered']))      
            country        = list(specificData['countryregion'])[0]      
            lat            = list(specificData['latitude'])[0]      
            long           = list(specificData['latitude'])[0]      

            dataHold.append([totalConfirmed,country,totalDeaths,lat,long,ps,totalrecovered,date])      
finData = pd.DataFrame(dataHold,columns = ['confirmed','countryregion','deaths','latitude','longitude','provincestate','recovered','timestamp'])      

In [73]:
print(len(data))
data = data.append(finData)
data

22553


,confirmed,countryregion,deaths,latitude,longitude,provincestate,recovered,timestamp
0,18.0,US,0.0,47.5289,-99.7840,North Dakota,0.0,1584661384
1,30.0,US,0.0,39.3185,-75.5071,Delaware,0.0,1584661384
2,31.0,US,1.0,38.4561,-92.2884,Missouri,0.0,1584661384
3,37.0,US,2.0,37.6681,-84.6701,Kentucky,0.0,1584661384
4,44.0,US,0.0,41.6809,-71.5118,Rhode Island,0.0,1584661384
...,...,...,...,...,...,...,...,...
313,64.0,Libya,3.0,26.3351,26.3351,Libya,24.0,1589155200
314,174.0,Togo,11.0,8.6195,8.6195,Togo,89.0,1589155200
315,2198.0,US,111.0,18.2208,18.2208,Puerto Rico,0.0,1589155200
316,1023.0,France,11.0,-12.8275,-12.8275,Mayotte,492.0,1589155200


In [74]:
with open("input/coronaUTD.csv","w",newline="\n") as f:
    data.to_csv(f,index=False,sep=",",header = True)